<a href="https://colab.research.google.com/github/KathirAvan250795/Learning-/blob/main/finetune_creditcard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.6 MB/s eta 0:00:00


In [2]:
!pip install GPUtil


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=2c8fe19baf32b299def1a9ceb515669a30e38bdf924415b6bcee7e2e885fb3fa
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [3]:
# Generated code may be subject to a license | Talkischeap/inedv

import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()


In [5]:
if "COLAB_GPU" in os.environ:
    !huggingface-cli login
else:
    notebook_login()


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Creditcard_Finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been sav

In [6]:
# Generated code may be subject to a license | Actable-AI/ml-utils | FreddyBanana/ChatGLM2-LoRA-Trainer

base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
train_dataset = load_dataset(
    "json",
    data_files="/content/final_dataset_clean4.jsonl",
    split="train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
def format_example(example):
    text = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    return {"text": text}

train_dataset = train_dataset.map(format_example)


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

In [15]:
print(train_dataset.column_names)
train_dataset[0]["text"]


['instruction', 'input', 'output', 'text']


'### Instruction:\nExtract the name of the credit card.\n\n### Input:\nWhat is the name of the credit card mentioned?\n\n### Response:\nAxis Bank Neo Credit Card'

In [10]:
train_dataset[8]


{'instruction': 'Extract the discount on Myntra purchases.',
 'input': 'What discount is available on Myntra purchases with the Axis Bank Neo Credit Card?',
 'output': 'INR 150 on purchases of a minimum of INR 999 on selected styles'}

In [11]:
tokenizer = LlamaTokenizer.from_pretrained(
    base_model_id,
    use_fast=False,
    trust_remote_code=True,
    add_eos_token=True
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [16]:
tokenized_train_dataset = train_dataset.map(
    lambda x: tokenizer(
        x["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    ),
    batched=True
)


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

In [24]:
tokenized_train_dataset = tokenized_train_dataset.map(
    lambda x: {"labels": x["input_ids"]},
    batched=True
)


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

In [26]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(
    ["instruction", "input", "output", "text"]
)


ValueError: Column name ['input', 'instruction', 'text', 'output'] not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']

In [27]:
print(tokenized_train_dataset.column_names)


['input_ids', 'attention_mask', 'labels']


In [21]:
tokenized_train_dataset[15]


{'instruction': 'Extract the discount on BookMyShow.',
 'input': 'What discount is available on BookMyShow with the Axis Bank Neo Credit Card?',
 'output': 'Rs. 1200 discount',
 'text': '### Instruction:\nExtract the discount on BookMyShow.\n\n### Input:\nWhat discount is available on BookMyShow with the Axis Bank Neo Credit Card?\n\n### Response:\nRs. 1200 discount',
 'input_ids': [1,
  835,
  2799,
  4080,
  29901,
  13,
  5647,
  1461,
  278,
  2313,
  792,
  373,
  6726,
  3421,
  8964,
  29889,
  13,
  13,
  2277,
  29937,
  10567,
  29901,
  13,
  5618,
  2313,
  792,
  338,
  3625,
  373,
  6726,
  3421,
  8964,
  411,
  278,
  319,
  11497,
  10253,
  2448,
  29877,
  24596,
  277,
  9160,
  29973,
  13,
  13,
  2277,
  29937,
  13291,
  29901,
  13,
  29934,
  29879,
  29889,
  29871,
  29896,
  29906,
  29900,
  29900,
  2313,
  792,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,

In [22]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Enable memory saving
model.gradient_checkpointing_enable()

# Prepare model for 4bit QLoRA training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,                    # Better learning capacity than 8
    lora_alpha=32,           # Standard stable value
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Inject LoRA adapters
model = get_peft_model(model, lora_config)

# Print trainable params (sanity check)
model.print_trainable_parameters()


trainable params: 39,976,960 || all params: 6,778,392,576 || trainable%: 0.5898


In [37]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Disable cache during training
model.config.use_cache = False

# Data collator for causal LM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="/content/llama2-lora-output",
    per_device_train_batch_size=1,      # MUST be 1 for 7B on Colab
    gradient_accumulation_steps=1,       # effective batch size = 4
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,                          # use fp16 on T4/V100
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="none",
    run_name="llama2-creditcard-lora",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    data_collator=data_collator
)

# 🚀 Start training
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


KeyboardInterrupt: 

In [35]:
!rm -rf /content/llama2-lora-output
